In [1]:
!pip install pillow

In [2]:
from PIL import Image  # Opens and edits images.
from IPython.display import display #Shows images inside Jupyter Notebook.

# Convert encoding data into 8-bit binary form
def genData(data):
    return [format(ord(i), '08b') for i in data]

# Modify pixels based on binary data
def modPix(pix, data):
    datalist = genData(data)
    lendata = len(datalist)
    imdata = iter(pix)

    for i in range(lendata):
        pix = [value for value in imdata.__next__()[:3] + 
               imdata.__next__()[:3] + 
               imdata.__next__()[:3]]

        for j in range(8):
            if datalist[i][j] == '0' and pix[j] % 2 != 0:
                pix[j] -= 1
            elif datalist[i][j] == '1' and pix[j] % 2 == 0:
                pix[j] = pix[j] - 1 if pix[j] != 0 else pix[j] + 1

        if i == lendata - 1:
            if pix[-1] % 2 == 0:
                pix[-1] = pix[-1] - 1 if pix[-1] != 0 else pix[-1] + 1
        else:
            if pix[-1] % 2 != 0:
                pix[-1] -= 1

        yield tuple(pix[0:3])
        yield tuple(pix[3:6])
        yield tuple(pix[6:9])

def encode_enc(newimg, data):
    w = newimg.size[0]
    (x, y) = (0, 0)
    for pixel in modPix(newimg.getdata(), data):
        newimg.putpixel((x, y), pixel)
        x = 0 if x == w - 1 else x + 1
        y += 1 if x == 0 else 0

# Encode data into an image
def encode():
    img = input("Enter image name (with extension): ")
    image = Image.open(img, 'r')
    display(image)  # Show image inside Jupyter Notebook

    data_source = input("Do you want to enter text (T) or use a file (F)? ").strip().upper()
    
    if data_source == "F":
        file_name = input("Enter the text file name (with extension): ")
        try:
            with open(file_name, 'r', encoding='utf-8') as f:
                data = f.read().strip()
        except FileNotFoundError:
            print("File not found! Exiting.")
            return
    else:
        data = input("Enter data to be encoded: ")

    if len(data) == 0:
        raise ValueError("Data is empty")

    password = input("Set a password for this message: ").strip()
    data = password + "::" + data  # Embed password in the message

    newimg = image.copy()
    encode_enc(newimg, data)

    new_img_name = input("Enter the name of new image (with extension): ")
    newimg.save(new_img_name)
    display(newimg)  # Show encoded image inside Jupyter Notebook
    print(f"Data encoded successfully in {new_img_name}")

# Decode the hidden message
def decode():
    img = input("Enter image name (with extension): ")
    image = Image.open(img, 'r')

    data = ''
    imgdata = iter(image.getdata())

    while True:
        pixels = [value for value in imgdata.__next__()[:3] +
                  imgdata.__next__()[:3] +
                  imgdata.__next__()[:3]]

        binstr = ''.join(['0' if i % 2 == 0 else '1' for i in pixels[:8]])
        data += chr(int(binstr, 2))

        if pixels[-1] % 2 != 0:
            break

    entered_password = input("Enter the password to decrypt: ").strip()
    
    if "::" in data:
        stored_password, message = data.split("::", 1)
        if stored_password == entered_password:
            print("Decoded Message:", message)
        else:
            print("Incorrect password! Cannot reveal the message.")
    else:
        print("Error: No password found in the encoded message.")

# Main function
def incript():
    choice = input(":: Welcome to Steganography ::\n1. Encode\n2. Decode\nChoose: ")
    
    if choice == "1":
        encode()
    elif choice == "2":
        decode()
    else:
        print("Invalid choice!")


In [ ]:
incript()

In [ ]:
incript()